In [1]:
import sys, os, math
import numpy as np
sys.path.insert(0,os.path.expanduser('~/git/pyzx')) # git version
sys.path.insert(0,'/workspaces/pyzx')
import pyzx as zx
from pyzx import print_matrix
from pyzx.basicrules import *
import time
from fractions import Fraction
import random
from pyzx.gflow import gflow
from pyzx.pauliwebs import preprocess
from pyzx.utils import vertex_is_zx
from pyzx.graph.base import BaseGraph
from pyzx import VertexType, EdgeType
# zx.settings.drawing_backend = 'matplotlib'

Z = zx.VertexType.Z
X = zx.VertexType.X
B = zx.VertexType.BOUNDARY
SE = zx.EdgeType.SIMPLE
HE = zx.EdgeType.HADAMARD

In [2]:
def transpose(p):
    pt = dict()
    for k,s in p.items():
        for v in s:
            if v not in pt: pt[v] = set()
            pt[v].add(k)
    return pt
def extend(g, s):
    s1 = s.copy()
    for v in s:
        s1 |= {v for v in g.neighbors(v) if vertex_is_zx(g.type(v))}
    return s1
def extend_dict(g, p):
    pe = dict()
    for k,s in p.items():
        pe[k] = extend(g, s)
    return pe
def is_dag(p):
    pt = transpose(p)
    for k,s in p.items():
        for v in s:
            if k != v and k in pt and v in pt[k]:
                return False
    return True

def is_clifford(p):
    return p in (0, 1, Fraction(1, 2), Fraction(3,2))

def gadgetize(g: BaseGraph):
    for v in list(g.vertices()):
        p = g.phase(v)
        if not is_clifford(p) and g.vertex_degree(v) > 1:
            x = g.add_vertex(VertexType.Z, -1, g.row(v))
            y = g.add_vertex(VertexType.Z, -2, g.row(v))
            g.add_edge((x, y), EdgeType.HADAMARD)
            g.add_edge((v, x), EdgeType.HADAMARD)
            g.set_phase(y, p)
            g.set_phase(v, 0)


In [3]:
random.seed(1330)
g = zx.generate.cliffordT(3, 60, p_t=0.2)
zx.draw(g)
g

Graph(94 vertices, 113 edges)

In [4]:
g1 = g.copy()
zx.full_reduce(g1)
gadgetize(g1)
g1.normalize()
g1 = g1.copy()
preprocess(g1)
#zx.to_rg(g1)
# g2 = g1.copy()
# for e in list(g2.edges()): zx.hrules.had_edge_to_hbox(g2, e)
zx.draw(g1, labels=True)

In [11]:
gf = gflow(g1, focus=True, pauli=True)
if gf:
    o,p = gf
    vs = [v for v in g1.vertices() if not is_clifford(g1.phase(v))]
    list.sort(vs, key=lambda v: o[v], reverse=True)
    for v in vs: print(f"{v} <- {p[v]} + {extend(g1, p[v]).difference(p[v])}")

15 <- {11, 28, 7} + {8, 10, 15, 29, 30}
23 <- {11, 22} + {8, 30, 23}
25 <- {24, 31} + {32, 9, 25}
27 <- {26, 28} + {10, 27, 29}
17 <- {10, 11, 15, 16, 27, 29, 31} + {32, 3, 7, 8, 9, 17, 26, 28, 30}
21 <- {8, 10, 11, 20, 23, 27, 28, 29, 30, 31} + {32, 3, 5, 7, 9, 21, 22, 26}
19 <- {32, 5, 8, 9, 11, 15, 18, 21, 23, 25, 30, 31} + {3, 4, 6, 7, 10, 19, 20, 22, 24}


In [12]:
gf = gflow(g1, focus=True, reverse=False, pauli=True)
if gf:
    o,p = gf
    pt = transpose(p)
    print(o)
    print(pt)

{32: 4, 29: 4, 30: 4, 8: 3, 9: 3, 10: 3, 11: 3, 15: 3, 23: 3, 25: 3, 27: 3, 28: 3, 31: 3, 3: 2, 5: 2, 7: 2, 17: 2, 21: 2, 22: 2, 24: 2, 26: 2, 4: 1, 6: 1, 16: 1, 19: 1, 20: 1, 18: 0}
{11: {3, 4, 5, 8, 15, 17, 19, 21, 23}, 31: {3, 4, 5, 9, 17, 19, 21, 25}, 28: {5, 10, 15, 21, 27}, 30: {4, 5, 11, 19, 21}, 7: {15}, 22: {23}, 24: {25}, 26: {27}, 29: {3, 5, 6, 17, 21, 28}, 32: {19, 4, 31}, 10: {3, 5, 6, 17, 21}, 15: {3, 4, 6, 7, 17, 19}, 27: {3, 5, 6, 17, 21, 26}, 8: {21, 19, 4, 5}, 23: {4, 5, 19, 21, 22}, 16: {17}, 20: {21}, 25: {24, 19, 4}, 5: {19, 4, 6}, 9: {19, 4}, 21: {19, 4, 20, 6}, 17: {16}, 18: {19}, 19: {18}}
